# DH - Eprene — Reproducible Analyses

This notebook reproduces key visuals: timelines, heatmap, and agency analysis.

In [ ]:

import pandas as pd, matplotlib.pyplot as plt
facts = pd.read_csv('../data/nature_philosophy_material_facts.csv')
facts["Century"] = (facts["Year"]//100+1).astype(int)
display(facts.head())

plt.figure(figsize=(10,6))
for cat in facts["Category"].unique():
    sub = facts[facts["Category"]==cat].sort_values("Year")
    plt.plot(sub["Year"], range(1,len(sub)+1), marker='o', label=cat)
plt.legend(); plt.title("Cumulative Philosophical vs Material Events"); plt.xlabel("Year"); plt.ylabel("Cumulative")
plt.show()


In [ ]:

import pandas as pd, matplotlib.pyplot as plt
hm = pd.read_csv('../data/nature_poetry_insights_heatmap.csv')
indicators = [c for c in hm.columns if c not in ['motif','severity_1to5']]
plt.figure(figsize=(10,6))
plt.imshow(hm[indicators].values, aspect='auto')
plt.yticks(range(len(hm)), hm['motif'])
plt.xticks(range(len(indicators)), indicators, rotation=45, ha='right')
plt.title('Motif × Stressor Matrix'); plt.tight_layout(); plt.show()


In [ ]:

import re, pandas as pd, matplotlib.pyplot as plt
from lxml import etree
tei_path = '../texts/nature_two_states_corpus.xml'
ns = {"t": "http://www.tei-c.org/ns/1.0"}
tree = etree.parse(tei_path)
texts = tree.xpath("//t:text//t:body//t:p | //t:text//t:body//t:lg//t:l", namespaces=ns)
content = []
for node in texts:
    txt = " ".join("".join(node.itertext()).split())
    if txt: content.append(txt)
text = "\n".join(content)

nature_nouns = r"(river|rivers|wind|winds|forest|forests|bird|birds|sea|seas|sky|skies|mountain|mountains|tree|trees|lake|lakes|ocean|oceans)"
active_verbs = r"(sings?|flows?|whispers?|moves?|breathes?|teaches?|rises?|falls?|glows?|blooms?|calls?|speaks?)"
passive_patterns = r"(is|was|were|are|be|been|being)\s+(cut|felled|drained|polluted|measured|monitored|exploited|mined|dredged|dammed|logged)"

sentences = re.split(r"(?<=[.!?])\s+", text.lower())
active = sum(1 for s in sentences if re.search(rf"\b{nature_nouns}\b.*\b{active_verbs}\b", s))
passive = sum(1 for s in sentences if re.search(rf"\b{nature_nouns}\b.*\b{passive_patterns}\b", s))

print({'nature_as_subject_active': active, 'nature_as_object_passive': passive})
plt.bar(['Nature as Subject (active)','Nature as Object (passive)'], [active, passive]); plt.title('Heuristic Agency'); plt.show()
